In [2]:
import requests
import json
import time

In [3]:
# Get BEARER_TOKEN from key.json
with open('../../key.json') as f:
	key = json.load(f)
	bearer_token = key['BEARER_TOKEN']

In [4]:
# Json path
json_path = '../../GMouYes/data/userProfile.json'
# Open json file
json_file = open(json_path, 'r')
# Load json file
json_data = json.load(json_file)

In [5]:
id_list = []
for i in json_data:
	id_list.append(i)
print(id_list)

['1132988843609808899', '153701624', '242877245', '3131222121', '4721076616', '47297687', '707966804253999105', '717712395112751104', '85810276', '937064669608628226']


In [ ]:
# Get timeline
def get_timeline(user_id, num_tweets = 5):
	# Create the url
	url = 'https://api.twitter.com/2/users/' + user_id + '/tweets?max_results=' + str(num_tweets)
	# Create request headers
	headers = {
		'authorization': 'Bearer ' + bearer_token,
		'user-agent': 'v2FilteredStreamPython',
	}
	# Make the request
	response = requests.get(url, headers=headers)
	return response.json()

In [ ]:
users_timeline = {}
for i in id_list:
	# Get the timeline 
	timeline = get_timeline(i, 100)
	# Add the timeline to the dictionary
	users_timeline[i] = timeline
# Write the dictionary to a json file
with open('users_timeline.json', 'w') as f:
	json.dump(users_timeline, f)

In [12]:
print(json.dumps(users_timeline, indent=4, sort_keys=True))

NameError: name 'users_timeline' is not defined

In [10]:
# Get timeline
get_followers_counter = 0
def get_followers(user_id):
    # Create the url
    return_json = {}
    url = 'https://api.twitter.com/2/users/{}/followers'.format(user_id)
    while True:
        # Create request headers
        headers = {
            'authorization': 'Bearer ' + bearer_token,
            'user-agent': 'v2FilteredStreamPython',
        }
        # Make the request
        global get_followers_counter
        get_followers_counter += 1
        if get_followers_counter >= 15:
            get_followers_counter = 0
            # Sleep for 15 minutes
            print('Sleeping for 15 minutes')
            time.sleep(60 * 15)
        response = requests.get(url, headers=headers).json()
        # Add up the response count
        try:
            return_json["meta"]["result_count"] = response["meta"]["result_count"] + return_json["meta"]["result_count"]
        except:
            return_json["meta"] = response["meta"]
        # Merge the response with the return_json
        try:
            for i in response["data"]:
                return_json["data"].append(i)
        except:
            return_json["data"] = response["data"]

      
        # Check if next_token exists in response
        try:
            # Update the url with the next_token
            url = 'https://api.twitter.com/2/users/{}/followers?pagination_token={}'.format(
                user_id, response["meta"]["next_token"])
        except:
            # If next_token does not exist, break the loop
            break
    return return_json


In [11]:
followers = {}
for i in id_list:
	followers[i] = get_followers(i)

# Write the dictionary to a json file
with open('../../data/followers.json', 'w') as f:
	json.dump(followers, f)
print(json.dumps(followers, indent=4, sort_keys=True))

Sleeping for 15 minutes
